In [15]:
import os
import pickle
import pandas as pd
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
from numpy import nan

In [16]:
files = [x for x in os.listdir('heatmap_ERF/') if 'results' in x]

In [17]:
all_results = []
for file in files:
    with open('heatmap_ERF/'+file, 'rb') as f:
        result_file = pickle.load(f)
        for r in result_file['results']:
            all_results.append(r)
            
result_df = pd.DataFrame(all_results)

In [18]:
with open('heatmap_ERF/paramgrid_newparams_a03.txt','rb') as f:
    param_grid = pickle.load(f)

In [20]:
all_results

[]

In [11]:
along_m = sorted(list(set(result_df['m'])))
along_n = sorted(list(set(result_df['n'])))
along_ap = sorted(list(set(result_df['ap'])))
along_ttr = sorted(list(set(result_df['ttr'])))
along_qd = sorted(list(set(result_df['qd'])))
along_m.remove(20000)
along_ap.remove(7.8)
along_ttr.remove(5)
along_qd.remove(1-3e-6)
along_n.remove(87)
all_along_params = [('m',along_m), ('n',along_n), ('ap',along_ap), ('ttr',along_ttr), ('qd',along_qd)]
all_names_params = ['m', 'n', 'ap', 'ttr', 'qd']

KeyError: 'm'

In [12]:
all_colormap_params = list(combinations(all_along_params, 2))
all_colormap_names = list(combinations(all_names_params,2))

NameError: name 'all_along_params' is not defined

In [10]:
dict_other_vals = {'m':20000, 'n':87, 'ap': 7.8, 'qd':1-3e-6, 'ttr':5}

#### 1) Generating all matrices for colormaps

In [ ]:
all_matrices_for_cmap = []

for comb in all_colormap_params:
    params_axes = (comb[0][0], comb[1][0])
    print(params_axes)
    other_params = set(all_names_params)-set(params_axes)

    colormap_params = np.zeros((len(comb[0][1]), len(comb[1][1])))
    
    for xind in range(len(comb[0][1])):
        for yind in range(len(comb[0][1])):
            df_axes = result_df.copy()
            for p in other_params:
                df_axes = df_axes[df_axes[p] == dict_other_vals[p]]
            df_axes = df_axes[(df_axes[params_axes[0]] == comb[0][1][xind])&(df_axes[params_axes[1]] == comb[1][1][yind])]
            colormap_params[xind, yind] = list(df_axes['Y_treatment'])[0]/list(df_axes['Y_control'])[0]
            
    all_matrices_for_cmap.append(colormap_params)

#### 2) Plotting all 10 cmaps

##### Indices for plots

In [6]:
indices_colormaps = []
for i in range(5):
    for j in range(2):
        indices_colormaps.append((i,j))
indices_colormaps = dict(zip(range(10), indices_colormaps))

##### Labels for axes

In [7]:
def calculate_ticks(ticklabels, truerange, tickrange):
    tick_pos = []
    for l in ticklabels:
        NewValue = (((l - truerange[0]) * (tickrange[1] - tickrange[0])) / (truerange[1] - truerange[0])) + tickrange[0]
        tick_pos.append(NewValue)
    return(np.array(tick_pos))

def calculate_ticks_log(ticklabels, truerange, tickrange):
    tick_labels = np.log(ticklabels)
    true_range = np.log(truerange)
    tick_pos = []
    for l in ticklabels:
        NewValue = (((l - truerange[0]) * (tickrange[1] - tickrange[0])) / (truerange[1] - truerange[0])) + tickrange[0]
        tick_pos.append(NewValue)
    return(np.array(tick_pos))

In [8]:
param_ticks = {'m':[1500,5000,10000,15000,20000,25000,29900],
               'n':[1,200,400,600,800,1000],
               'ap':[7.7,8.0,8.3,8.6,8.8],
               'qd':[1-1e-6,1-3e-6,1-5e-6,1-7.5e-6,1-1e-5], 
               'ttr':[0,3,7,10,14]}
param_ranges = {'m':[1500,29900],
                'n':[1,1000],
                'ap':[7.7,8.8],
                'qd':[1e-6,1e-5],
                'ttr':[0,14]}

In [ ]:
calculate_ticks_log(1-np.array(param_ticks['qd']), param_ranges['qd'], [0,19])

In [ ]:
all_xticks, all_yticks = [],[]
all_xticklabels, all_yticklabels = [],[]

for i in range(10):
    
    x_param, y_param = all_colormap_names[i]

    if y_param == 'qd':
        ticks_y = calculate_ticks_log(1-np.array(param_ticks[y_param]), param_ranges[y_param], [0,19])
    else:
        ticks_y = calculate_ticks(np.array(param_ticks[y_param]), param_ranges[y_param], [0,19])
        
    if y_param == 'qd':
        labels_y = list(["{:.1e}".format(1-x) for x in np.array(param_ticks[y_param])])
    elif y_param == 'ap':
        labels_y = [np.round(x,2) for x in param_ticks[y_param]]
    else:
        labels_y = [int(np.round(x)) for x in param_ticks[y_param]]
        
    if x_param == 'qd': 
        ticks_x = 19-np.array(calculate_ticks_log(1-np.array(param_ticks[x_param]), param_ranges[x_param], [0,19])[::-1])
    else:
        ticks_x = 19-np.array(calculate_ticks(np.array(param_ticks[x_param]), param_ranges[x_param], [0,19])[::-1])
            

    if x_param == 'qd':
        labels_x = list(["{:.1e}".format(1-x) for x in np.array(param_ticks[x_param])])
    elif x_param == 'ap':
        labels_x = [np.round(x,2) for x in param_ticks[x_param]]
    else:
        labels_x = [int(np.round(x)) for x in param_ticks[x_param]]
    
    
    all_xticks.append(ticks_x)
    all_yticks.append(ticks_y)
    all_xticklabels.append(labels_x)
    all_yticklabels.append(labels_y)

In [ ]:
params_description = {'m':'Number of lethal positions $m$',
                      'n':'Number of gain-of-function positions $n$',
                      'ap': 'Clearance rate in clearance phase $a_1$',
                      'qd': 'Viral mutation rate under treatment $u_1$',
                      'ttr': 'Time of start of treatment'}

In [ ]:
fig, ax = plt.subplots(5,2,figsize=(8,18))

for i in range(10):
    im = ax[indices_colormaps[i]].imshow(all_matrices_for_cmap[i][::-1], cmap = 'seismic', vmin = 0., vmax = 2)
    if all_colormap_params[i][1][0] in ['qd']:
        im = ax[indices_colormaps[i]].imshow(np.flip(all_matrices_for_cmap[i][::-1],1), cmap = 'seismic', vmin = 0., vmax = 2)

    ax[indices_colormaps[i]].set_xticks(all_yticks[i])
    ax[indices_colormaps[i]].set_xticklabels(all_yticklabels[i])
    
    ax[indices_colormaps[i]].set_yticks(all_xticks[i])
    ax[indices_colormaps[i]].set_yticklabels(all_xticklabels[i][::-1])    
    
    
    ax[indices_colormaps[i]].set_ylabel(params_description[all_colormap_params[i][0][0]])
    ax[indices_colormaps[i]].set_xlabel(params_description[all_colormap_params[i][1][0]])
    
    
plt.tight_layout(pad=1.0)
plt.savefig('colormaps_all_params_FEB12.jpg', dpi = 600)

#### Splitting into main and supplementary

In [ ]:
indices_colormaps_maintext = []
for i in range(3):
    for j in range(2):
        indices_colormaps_maintext.append((i,j))
indices_colormaps_maintext = dict(zip([1,2,3,9], indices_colormaps_maintext))

#### making black line on colormap

In [ ]:
def sorcerers_stone(T, b, a1, q0, q1, m, n):
    notreat = (np.exp(b*T*(1-q0**n)*(q0**m))/(a1-b*q0**m)) - (1/(a1-b*q0**(m+n)))
    treat = (np.exp(b*T*(1-q0**n)*(q0**m))/(a1-b*q1**m)) - (1/(a1-b*q1**(m+n)))
    return(treat/notreat)

def convert_values(val, OldRange, NewRange):
    OldR = OldRange[1]-OldRange[0]
    NewR = NewRange[1]-NewRange[0]
    NewVal = (((val-OldRange[0])*(NewR))/OldR)+NewRange[0]
    return(NewVal)

In [ ]:
black_line_0 = []

for n_val in along_n:
    formula = sorcerers_stone(5, 7.61, 7.8, 1-1e-6, 1-3e-6, np.array(np.linspace(1500,29900,1000)), n_val)
    index_2 = np.searchsorted(-formula, -1)
    formula_m = np.array(np.linspace(1500,29900,1000))[index_2]
    black_line_0.append(formula_m)
    
black_line_1 = []

for a1_val in along_ap:
    formula = sorcerers_stone(5, 7.6, a1_val, 1-1e-6, 1-3e-6, np.array(np.linspace(1500,29900,1000)), 87)
    index_2 = np.searchsorted(-formula, -1)
    formula_m = np.array(np.linspace(1500,29900,1000))[index_2]
    black_line_1.append(formula_m)
    
black_line_2 = []

for q1_val in along_qd:
    formula = sorcerers_stone(5, 7.6, 7.8, 1-1e-6, q1_val, np.array(np.linspace(1500,29900,1000)), 87)
    index_2 = np.searchsorted(-formula, -1)
    formula_m = np.array(np.linspace(1500,29900,1000))[index_2]
    black_line_2.append(formula_m)
    
    
black_line_3 = []

for n_val in along_n:
    formula = sorcerers_stone(5, 7.6, np.array(np.linspace(7.7,8.8,1000)), 1-1e-6, 1-3e-6, 20000, n_val)
    index_2 = np.searchsorted(-formula, -1)
    formula_m = np.array(np.linspace(7.7,8.8,1000))[index_2]
    black_line_3.append(formula_m)

In [ ]:
black_line_0_convert = convert_values(np.array(black_line_0), [1500,29900], [0,19])
black_line_1_convert = convert_values(np.array(black_line_1), [1500,29900], [0,19])
black_line_2_convert = convert_values(np.array(black_line_2), [1500,29900], [0,19])
black_line_3_convert = convert_values(np.array(black_line_3), [7.7,8.8], [0,19])

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(8,8))

for i in [1,2,3,9]:
    im = ax[indices_colormaps_maintext[i]].imshow(all_matrices_for_cmap[i][::-1], cmap = 'seismic', vmin = 0., vmax = 2)
    if all_colormap_params[i][1][0] in ['qd']:
        im = ax[indices_colormaps_maintext[i]].imshow(np.flip(all_matrices_for_cmap[i][::-1],1), cmap = 'seismic', vmin = 0., vmax = 2)

    ax[indices_colormaps_maintext[i]].set_xticks(all_yticks[i])
    ax[indices_colormaps_maintext[i]].set_xticklabels(all_yticklabels[i])
    
    ax[indices_colormaps_maintext[i]].set_yticks(all_xticks[i])
    ax[indices_colormaps_maintext[i]].set_yticklabels(all_xticklabels[i][::-1])    
    
    
    ax[indices_colormaps_maintext[i]].set_ylabel(params_description[all_colormap_params[i][0][0]])
    ax[indices_colormaps_maintext[i]].set_xlabel(params_description[all_colormap_params[i][1][0]])
    
    

ax[0,0].plot(np.arange(0,20), 19-black_line_1_convert, c = 'k')
ax[1,0].plot(np.arange(1,20), 19-black_line_2_convert[:-1][::-1], c = 'k')
    
plt.tight_layout(pad=1.0)
plt.savefig('colormap_newaxlabels_FEB12.jpg', dpi = 400, bbox_inches = 'tight')

In [ ]:
indices_colormaps_supplementary = []
for i in range(3):
    for j in range(2):
        indices_colormaps_supplementary.append((i,j))
indices_colormaps_supplementary = dict(zip([0,4,5,6,7,8], indices_colormaps_supplementary))

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(8,11))

for i in [0,4,5,6,7,8]:
    im = ax[indices_colormaps_supplementary[i]].imshow(all_matrices_for_cmap[i][::-1], cmap = 'seismic', vmin = 0., vmax = 2)
    if all_colormap_params[i][1][0] in ['qd']:
        im = ax[indices_colormaps_supplementary[i]].imshow(np.flip(all_matrices_for_cmap[i][::-1],1), cmap = 'seismic', vmin = 0., vmax = 2)

    ax[indices_colormaps_supplementary[i]].set_xticks(all_yticks[i])
    ax[indices_colormaps_supplementary[i]].set_xticklabels(all_yticklabels[i])
    
    ax[indices_colormaps_supplementary[i]].set_yticks(all_xticks[i])
    ax[indices_colormaps_supplementary[i]].set_yticklabels(all_xticklabels[i][::-1])    
    
    
    ax[indices_colormaps_supplementary[i]].set_ylabel(params_description[all_colormap_params[i][0][0]])
    ax[indices_colormaps_supplementary[i]].set_xlabel(params_description[all_colormap_params[i][1][0]])
    
ax[0,0].plot(np.arange(0,20), 19-black_line_0_convert, c = 'k')

plt.tight_layout(pad=1.0)
plt.savefig('colormap_supp_newaxlabels_FEB12.jpg', dpi = 400, bbox_inches = 'tight')